In [46]:
# import libraries
import requests
import pandas as pd
import numpy as np
import math
import pickle
from pandas import json_normalize 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# for regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# for metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score 

# for plots 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import pairplot

In [47]:
# season stats per team
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/TeamSeasonStats/2020REG?key=d8b5ea01537141eb9a320f95994b7109')

# put in dataframe
team_stats_2020 = pd.DataFrame.from_dict(response.json())

# calculate average score
team_stats_2020['average_score'] = round((team_stats_2020['Score'] / team_stats_2020['Games']), 2)

# display first few
team_stats_2020.head()

# this one has a lot of data, it is condensed in this view (for whole thing check here: https://api.sportsdata.io/v3/nfl/scores/json/TeamSeasonStats/2020REG?key=d8b5ea01537141eb9a320f95994b7109)

,SeasonType,Season,Team,Score,OpponentScore,TotalScore,Temperature,Humidity,WindSpeed,OverUnder,PointSpread,ScoreQuarter1,ScoreQuarter2,ScoreQuarter3,ScoreQuarter4,ScoreOvertime,TimeOfPossession,FirstDowns,FirstDownsByRushing,FirstDownsByPassing,FirstDownsByPenalty,OffensivePlays,OffensiveYards,OffensiveYardsPerPlay,Touchdowns,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,PassingAttempts,PassingCompletions,PassingYards,PassingTouchdowns,PassingInterceptions,PassingYardsPerAttempt,PassingYardsPerCompletion,CompletionPercentage,PasserRating,ThirdDownAttempts,ThirdDownConversions,ThirdDownPercentage,FourthDownAttempts,FourthDownConversions,FourthDownPercentage,RedZoneAttempts,RedZoneConversions,GoalToGoAttempts,GoalToGoConversions,ReturnYards,Penalties,PenaltyYards,Fumbles,FumblesLost,TimesSacked,TimesSackedYards,QuarterbackHits,TacklesForLoss,Safeties,Punts,PuntYards,PuntAverage,Giveaways,Takeaways,TurnoverDifferential,OpponentScoreQuarter1,OpponentScoreQuarter2,OpponentScoreQuarter3,OpponentScoreQuarter4,OpponentScoreOvertime,OpponentTimeOfPossession,OpponentFirstDowns,OpponentFirstDownsByRushing,OpponentFirstDownsByPassing,OpponentFirstDownsByPenalty,OpponentOffensivePlays,OpponentOffensiveYards,OpponentOffensiveYardsPerPlay,OpponentTouchdowns,OpponentRushingAttempts,OpponentRushingYards,OpponentRushingYardsPerAttempt,OpponentRushingTouchdowns,OpponentPassingAttempts,OpponentPassingCompletions,OpponentPassingYards,OpponentPassingTouchdowns,OpponentPassingInterceptions,OpponentPassingYardsPerAttempt,OpponentPassingYardsPerCompletion,OpponentCompletionPercentage,OpponentPasserRating,OpponentThirdDownAttempts,OpponentThirdDownConversions,OpponentThirdDownPercentage,OpponentFourthDownAttempts,OpponentFourthDownConversions,OpponentFourthDownPercentage,OpponentRedZoneAttempts,OpponentRedZoneConversions,OpponentGoalToGoAttempts,OpponentGoalToGoConversions,OpponentReturnYards,OpponentPenalties,OpponentPenaltyYards,OpponentFumbles,OpponentFumblesLost,OpponentTimesSacked,OpponentTimesSackedYards,OpponentQuarterbackHits,OpponentTacklesForLoss,OpponentSafeties,OpponentPunts,OpponentPuntYards,OpponentPuntAverage,OpponentGiveaways,OpponentTakeaways,OpponentTurnoverDifferential,RedZonePercentage,GoalToGoPercentage,QuarterbackHitsDifferential,TacklesForLossDifferential,QuarterbackSacksDifferential,TacklesForLossPercentage,QuarterbackHitsPercentage,TimesSackedPercentage,OpponentRedZonePercentage,OpponentGoalToGoPercentage,OpponentQuarterbackHitsDifferential,OpponentTacklesForLossDifferential,OpponentQuarterbackSacksDifferential,OpponentTacklesForLossPercentage,OpponentQuarterbackHitsPercentage,OpponentTimesSackedPercentage,Kickoffs,KickoffsInEndZone,KickoffTouchbacks,PuntsHadBlocked,PuntNetAverage,ExtraPointKickingAttempts,ExtraPointKickingConversions,ExtraPointsHadBlocked,ExtraPointPassingAttempts,ExtraPointPassingConversions,ExtraPointRushingAttempts,ExtraPointRushingConversions,FieldGoalAttempts,FieldGoalsMade,FieldGoalsHadBlocked,PuntReturns,PuntReturnYards,KickReturns,KickReturnYards,InterceptionReturns,InterceptionReturnYards,OpponentKickoffs,OpponentKickoffsInEndZone,OpponentKickoffTouchbacks,OpponentPuntsHadBlocked,OpponentPuntNetAverage,OpponentExtraPointKickingAttempts,OpponentExtraPointKickingConversions,OpponentExtraPointsHadBlocked,OpponentExtraPointPassingAttempts,OpponentExtraPointPassingConversions,OpponentExtraPointRushingAttempts,OpponentExtraPointRushingConversions,OpponentFieldGoalAttempts,OpponentFieldGoalsMade,OpponentFieldGoalsHadBlocked,OpponentPuntReturns,OpponentPuntReturnYards,OpponentKickReturns,OpponentKickReturnYards,OpponentInterceptionReturns,OpponentInterceptionReturnYards,SoloTackles,AssistedTackles,Sacks,SackYards,PassesDefended,FumblesForced,FumblesRecovered,FumbleReturnYards,FumbleReturnTouchdowns,InterceptionReturnTouchdowns,BlockedKicks,PuntReturnTouchdowns,PuntReturnLong,KickReturnTouchdowns,KickReturnLong,BlockedKickReturnYards,BlockedKickReturnTouchdowns,FieldGoalReturnYards,Fie

In [3]:
# season standing per team
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/Standings/2020REG?key=d8b5ea01537141eb9a320f95994b7109')

# put in dataframe
team_standings_2020 = pd.DataFrame.from_dict(response.json())

# display first few
team_standings_2020.head()

,SeasonType,Season,Conference,Division,Team,Name,Wins,Losses,Ties,Percentage,PointsFor,PointsAgainst,NetPoints,Touchdowns,DivisionWins,DivisionLosses,ConferenceWins,ConferenceLosses,TeamID,DivisionTies,ConferenceTies,GlobalTeamID,DivisionRank,ConferenceRank
0,1,2020,AFC,East,BUF,Buffalo Bills,7,3,0,0.7,272,265,7,31,4,0,5,2,4,0,0,4,0,0
1,1,2020,AFC,East,MIA,Miami Dolphins,6,4,0,0.6,264,202,62,28,1,2,3,3,19,0,0,19,0,0
2,1,2020,AFC,East,NE,New England Patriots,4,6,0,0.4,209,238,-29,23,2,1,4,4,21,0,0,21,0,0
3,1,2020,AFC,East,NYJ,New York Jets,0,10,0,0.0,149,302,-153,15,0,4,0,8,24,0,0,24,0,0
4,1,2020,AFC,North,PIT,Pittsburgh Steelers,10,0,0,1.0,298,174,124,36,3,0,7,0,28,0,0,28,0,0


#### From Season Stats I can see us taking Score, Opponent Score, Avg Score and PAssing Dropbacks
#### From Standing we would take Wins, Division Wins, Conference Wins (maybe corresponding losses)


In [5]:
response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/ScoresByWeek/2020REG/10?key=d8b5ea01537141eb9a320f95994b7109')
current_week_games = pd.DataFrame.from_dict(response.json())
current_week_games = current_week_games[['AwayTeam','HomeTeam']]
current_week_games
for index, row in current_week_games.iterrows():
    print(row.AwayTeam)

IND
HOU
WAS
JAX
PHI
CIN
TB
DEN
NYJ
BUF
SEA
SF
BAL
MIN
LAC


In [95]:

response = requests.get('https://api.sportsdata.io/v3/nfl/scores/json/GameStats/2019REG?key=d8b5ea01537141eb9a320f95994b7109')
team_games_2019 = pd.DataFrame.from_dict(response.json())
team_games_2019.head()


,GameKey,Date,SeasonType,Season,Week,Stadium,PlayingSurface,Temperature,Humidity,WindSpeed,AwayTeam,HomeTeam,AwayScore,HomeScore,TotalScore,OverUnder,PointSpread,AwayScoreQuarter1,AwayScoreQuarter2,AwayScoreQuarter3,AwayScoreQuarter4,AwayScoreOvertime,HomeScoreQuarter1,HomeScoreQuarter2,HomeScoreQuarter3,HomeScoreQuarter4,HomeScoreOvertime,AwayTimeOfPossession,HomeTimeOfPossession,AwayFirstDowns,HomeFirstDowns,AwayFirstDownsByRushing,HomeFirstDownsByRushing,AwayFirstDownsByPassing,HomeFirstDownsByPassing,AwayFirstDownsByPenalty,HomeFirstDownsByPenalty,AwayOffensivePlays,HomeOffensivePlays,AwayOffensiveYards,HomeOffensiveYards,AwayOffensiveYardsPerPlay,HomeOffensiveYardsPerPlay,AwayTouchdowns,HomeTouchdowns,AwayRushingAttempts,HomeRushingAttempts,AwayRushingYards,HomeRushingYards,AwayRushingYardsPerAttempt,HomeRushingYardsPerAttempt,AwayRushingTouchdowns,HomeRushingTouchdowns,AwayPassingAttempts,HomePassingAttempts,AwayPassingCompletions,HomePassingCompletions,AwayPassingYards,HomePassingYards,AwayPassingTouchdowns,HomePassingTouchdowns,AwayPassingInterceptions,HomePassingInterceptions,AwayPassingYardsPerAttempt,HomePassingYardsPerAttempt,AwayPassingYardsPerCompletion,HomePassingYardsPerCompletion,AwayCompletionPercentage,HomeCompletionPercentage,AwayPasserRating,HomePasserRating,AwayThirdDownAttempts,HomeThirdDownAttempts,AwayThirdDownConversions,HomeThirdDownConversions,AwayThirdDownPercentage,HomeThirdDownPercentage,AwayFourthDownAttempts,HomeFourthDownAttempts,AwayFourthDownConversions,HomeFourthDownConversions,AwayFourthDownPercentage,HomeFourthDownPercentage,AwayRedZoneAttempts,HomeRedZoneAttempts,AwayRedZoneConversions,HomeRedZoneConversions,AwayGoalToGoAttempts,HomeGoalToGoAttempts,AwayGoalToGoConversions,HomeGoalToGoConversions,AwayReturnYards,HomeReturnYards,AwayPenalties,HomePenalties,AwayPenaltyYards,HomePenaltyYards,AwayFumbles,HomeFumbles,AwayFumblesLost,HomeFumblesLost,AwayTimesSacked,HomeTimesSacked,AwayTimesSackedYards,HomeTimesSackedYards,AwaySafeties,HomeSafeties,AwayPunts,HomePunts,AwayPuntYards,HomePuntYards,AwayPuntAverage,HomePuntAverage,AwayGiveaways,HomeGiveaways,AwayTakeaways,HomeTakeaways,AwayTurnoverDifferential,HomeTurnoverDifferential,AwayKickoffs,HomeKickoffs,AwayKickoffsInEndZone,HomeKickoffsInEndZone,AwayKickoffTouchbacks,HomeKickoffTouchbacks,AwayPuntsHadBlocked,HomePuntsHadBlocked,AwayPuntNetAverage,HomePuntNetAverage,AwayExtraPointKickingAttempts,HomeExtraPointKickingAttempts,AwayExtraPointKickingConversions,HomeExtraPointKickingConversions,AwayExtraPointsHadBlocked,HomeExtraPointsHadBlocked,AwayExtraPointPassingAttempts,HomeExtraPointPassingAttempts,AwayExtraPointPassingConversions,HomeExtraPointPassingConversions,AwayExtraPointRushingAttempts,HomeExtraPointRushingAttempts,AwayExtraPointRushingConversions,HomeExtraPointRushingConversions,AwayFieldGoalAttempts,HomeFieldGoalAttempts,AwayFieldGoalsMade,HomeFieldGoalsMade,AwayFieldGoalsHadBlocked,HomeFieldGoalsHadBlocked,AwayPuntReturns,HomePuntReturns,AwayPuntReturnYards,HomePuntReturnYards,AwayKickReturns,HomeKickReturns,AwayKickReturnYards,HomeKickReturnYards,AwayInterceptionReturns,HomeInterceptionReturns,AwayInterceptionReturnYards,HomeInterceptionReturnYards,AwaySoloTackles,AwayAssistedTackles,AwayQuarterbackHits,AwayTacklesForLoss,AwaySacks,AwaySackYards,AwayPassesDefended,AwayFumblesForced,AwayFumblesRecovered,AwayFumbleReturnYards,AwayFumbleReturnTouchdowns,AwayInterceptionReturnTouchdowns,AwayBlockedKicks,AwayPuntReturnTouchdowns,AwayPuntReturnLong,AwayKickReturnTouchdowns,AwayKickReturnLong,AwayBlockedKickReturnYards,AwayBlockedKickReturnTouchdowns,AwayFieldGoalReturnYards,AwayFieldGoalReturnTouchdowns,AwayPuntNetYards,HomeSoloTackles,HomeAssistedTackles,HomeQuarterbackHits,HomeTacklesForLoss,HomeSacks,HomeSackYards,HomePassesDefended,HomeFumblesForced,HomeFumblesRecovered,HomeFumbleReturnYards,HomeFumbleReturnTouchdowns,HomeInterceptionReturnTouchdowns,HomeBlockedKicks,HomePuntReturnTouchdowns,HomePuntReturnLong,HomeKickReturnTouchdo

In [96]:
team_games_2019.dtypes

GameKey                              object
Date                                 object
SeasonType                            int64
Season                                int64
Week                                  int64
Stadium                              object
PlayingSurface                       object
Temperature                         float64
Humidity                              int64
WindSpeed                           float64
AwayTeam                             object
HomeTeam                             object
AwayScore                             int64
HomeScore                             int64
TotalScore                            int64
OverUnder                           float64
PointSpread                         float64
AwayScoreQuarter1                     int64
AwayScoreQuarter2                     int64
AwayScoreQuarter3                     int64
AwayScoreQuarter4                     int64
AwayScoreOvertime                     int64
HomeScoreQuarter1               

In [112]:
# create new data frame
teams_season_2019 =  pd.DataFrame()
for index, row in team_games_2019.iterrows():
    # ---------------------------------------update each teams score per week------------------------------------------
    teams_season_2019.loc[row.HomeTeam,row.Week] = row.HomeScore
    teams_season_2019.loc[row.AwayTeam,row.Week] = row.AwayScore
    
    teams_season_2019[row.Week] = teams_season_2019[row.Week].fillna(0)

    # ---------------------------------------update each teams games played -------------------------------------------
    if 'GamesPlayed' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] = 0
        teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['GamesPlayed'] = teams_season_2019['GamesPlayed'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] += 1
    teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] += 1
        

    # ----------------------------------------------update score to week------------------------------------------------
    if 'TotalScore' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalScore'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalScore'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalScore'] = teams_season_2019['TotalScore'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalScore'] += row.HomeScore 
    teams_season_2019.loc[row.AwayTeam,'TotalScore'] += row.AwayScore
    
    # get avg points scored up to a certain week
    update_avg_home_score = teams_season_2019.loc[row.HomeTeam,'TotalScore'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_avg_away_score = teams_season_2019.loc[row.AwayTeam,'TotalScore'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'avg_up_to_week_{0}'.format(row.Week + 1)] = update_avg_home_score 
    teams_season_2019.loc[row.AwayTeam,'avg_up_to_week_{0}'.format(row.Week + 1)] = update_avg_away_score
    
    teams_season_2019['avg_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['avg_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalScore'] / (row.Week))
    
    # ----------------------------------- store wins per team up to a certain week ----------------------------------
    if 'TotalWins' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalWins'] = teams_season_2019['TotalWins'].fillna(0)
    # check who won
    if row.HomeScore > row.AwayScore:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] += 1
    else:
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] += 1
    
    
    teams_season_2019.loc[row.HomeTeam,'wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019.loc[row.HomeTeam,'TotalWins']
    teams_season_2019.loc[row.AwayTeam,'wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019.loc[row.AwayTeam,'TotalWins']
    
    if row.Week >= 4:
        teams_season_2019['wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['wins_up_to_week_{0}'.format(row.Week + 1)].fillna(0)
    # ----------------------------------- store first downs per team up to certain week ----------------------------------

    if 'TotalFirstDowns' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalFirstDowns'] = teams_season_2019['TotalFirstDowns'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] += row.HomeFirstDowns 
    teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] += row.AwayFirstDowns
    
    # get avg points scored up to a certain week
    update_first_downs_home = teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_first_downs_away = teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'first_downs_up_to_week_{0}'.format(row.Week + 1)] = update_first_downs_home 
    teams_season_2019.loc[row.AwayTeam,'first_downs_up_to_week_{0}'.format(row.Week + 1)] = update_first_downs_away
    
    teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalFirstDowns'] / (row.Week))
    
    # ----------------------------------- calculate third down % per team up to certain week ----------------------------------

    if 'ThirdDownPercentage' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] = 0.0
        teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] = 0.0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['ThirdDownPercentage'] = teams_season_2019['ThirdDownPercentage'].fillna(0.0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] += row.HomeThirdDownPercentage 
    teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] += row.AwayThirdDownPercentage
    
    # get avg points scored up to a certain week
    update_third_down_percentage_home = teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_third_down_percentage_away = teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = update_third_down_percentage_home
    teams_season_2019.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = update_third_down_percentage_away
    
    teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['ThirdDownPercentage'] / (row.Week))
    
    
    # ----------------------------------- calculate average time of possession per team ----------------------------------
    
    away_top = row.AwayTimeOfPossession.split(':')
    home_top = row.HomeTimeOfPossession.split(':')
    away_top = float('{0}.{1}'.format(away_top[0],away_top[1]))
    home_top = float('{0}.{1}'.format(home_top[0],home_top[1]))
    
    
    if 'TotalTimeOfPossession' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] = 0.0
        teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] = 0.0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalTimeOfPossession'] = teams_season_2019['TotalTimeOfPossession'].fillna(0.0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] += home_top
    teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] += away_top
    
    # get avg points scored up to a certain week
    update_time_of_possession_home = teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_time_of_possession_away = teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = update_time_of_possession_home
    teams_season_2019.loc[row.AwayTeam,'time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = update_time_of_possession_away
    
    teams_season_2019['time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['time_of_possession_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalTimeOfPossession'] / (row.Week ))
    
    
teams_season_2019

,1,GamesPlayed,TotalScore,avg_up_to_week_2,TotalWins,wins_up_to_week_2,TotalFirstDowns,first_downs_up_to_week_2,ThirdDownPercentage,third_down_percentage_up_to_week_2,TotalTimeOfPossession,time_of_possession_up_to_week_2,2,avg_up_to_week_3,wins_up_to_week_3,first_downs_up_to_week_3,third_down_percentage_up_to_week_3,time_of_possession_up_to_week_3,3,avg_up_to_week_4,wins_up_to_week_4,first_downs_up_to_week_4,third_down_percentage_up_to_week_4,time_of_possession_up_to_week_4,4,avg_up_to_week_5,wins_up_to_week_5,first_downs_up_to_week_5,third_down_percentage_up_to_week_5,time_of_possession_up_to_week_5,5,avg_up_to_week_6,wins_up_to_week_6,first_downs_up_to_week_6,third_down_percentage_up_to_week_6,time_of_possession_up_to_week_6,6,avg_up_to_week_7,wins_up_to_week_7,first_downs_up_to_week_7,third_down_percentage_up_to_week_7,time_of_possession_up_to_week_7,7,avg_up_to_week_8,wins_up_to_week_8,first_downs_up_to_week_8,third_down_percentage_up_to_week_8,time_of_possession_up_to_week_8,8,avg_up_to_week_9,wins_up_to_week_9,first_downs_up_to_week_9,third_down_percentage_up_to_week_9,time_of_possession_up_to_week_9,9,avg_up_to_week_10,wins_up_to_week_10,first_downs_up_to_week_10,third_down_percentage_up_to_week_10,time_of_possession_up_to_week_10,10,avg_up_to_week_11,wins_up_to_week_11,first_downs_up_to_week_11,third_down_percentage_up_to_week_11,time_of_possession_up_to_week_11,11,avg_up_to_week_12,wins_up_to_week_12,first_downs_up_to_week_12,third_down_percentage_up_to_week_12,time_of_possession_up_to_week_12,12,avg_up_to_week_13,wins_up_to_week_13,first_downs_up_to_week_13,third_down_percentage_up_to_week_13,time_of_possession_up_to_week_13,13,avg_up_to_week_14,wins_up_to_week_14,first_downs_up_to_week_14,third_down_percentage_up_to_week_14,time_of_possession_up_to_week_14,14,avg_up_to_week_15,wins_up_to_week_15,first_downs_up_to_week_15,third_down_percentage_up_to_week_15,time_of_possession_up_to_week_15,15,avg_up_to_week_16,wins_up_to_week_16,first_downs_up_to_week_16,third_down_percentage_up_to_week_16,time_of_possession_up_to_week_16,16,avg_up_to_week_17,wins_up_to_week_17,first_downs_up_to_week_17,third_down_percentage_up_to_week_17,time_of_possession_up_to_week_17,17,avg_up_to_week_18,wins_up_to_week_18,first_downs_up_to_week_18,third_down_percentage_up_to_week_18,time_of_possession_up_to_week_18
CHI,9.0,16.0,494.0,9.0,8.0,0.0,515.0,28.0,981.5,34.7,482.04,28.57,28.0,18.5,1.0,29.5,41.00,27.835,54.0,30.333333,2.0,31.666667,62.866667,29.010000,28.0,29.75,3.0,31.00,60.675,30.5750,36.0,31.00,3.0,30.00,61.160,29.4940,0.0,25.833333,0.0,25.000000,50.966667,24.578333,43.0,33.000000,3.0,29.666667,55.783333,28.301667,28.0,32.285714,3.0,31.857143,56.657143,29.687143,24.0,31.250000,3.0,30.000000,53.912500,28.403750,35.0,31.666667,4.0,29.222222,51.133333,28.425556,12.0,29.700000,4.0,29.200000,52.140000,28.827000,33.0,30.000000,5.0,29.727273,51.609091,29.216364,42.0,31.000000,6.0,30.416667,54.525000,29.323333,54.0,32.769231,7.0,31.307692,58.100000,29.543077,23.0,32.071429,7.0,31.571429,59.521429,29.877143,9.0,30.533333,7.0,31.533333,60.373333,29.642667,36.0,30.8750,8.0,32.1875,61.34375,30.127500
GB,17.0,16.0,653.0,17.0,13.0,1.0,556.0,23.0,1003.1,28.9,502.78,31.30,36.0,26.5,2.0,29.0,43.30,32.950,47.0,33.333333,3.0,28.666667,41.700000,30.053333,47.0,36.75,3.0,35.00,54.875,30.5925,59.0,41.20,4.0,36.40,52.560,31.7600,40.0,41.000000,5.0,36.666667,53.416667,32.033333,73.0,45.571429,6.0,36.857143,60.642857,31.777143,54.0,46.625000,7.0,36.750000,66.387500,31.946250,19.0,43.555556,7.0,35.222222,62.866667,31.163333,42.0,43.400000,8.0,35.700000,64.270000,31.169000,0.0,39.454545,0.0,32.454545,58.427273,28.335455,14.0,40.727273,8.0,35.454545,59.481818,31.531818,54.0,41.833333,9.0,35.250000,62.291667,31.273333,35.0,41.307692,10.0,34.923077,62.630769,31.193846,36.0,40.928571,11.0,33.928571,62.278571,30.778571,40.0,40.866667,12.0,34.200000,62.253333,31.214667,40.0,40.8125,13.0,34.7500,62.69375,31.423750
CAR,47.0,16.0,594.0,47.0,5.0,0.0,580.0,36.0,902.4,78.9

# Tashi

In [113]:
# Create a copy of the original dataset
df = team_games_2019.copy()

# Sliced DataFrame:Df by column name
df = df[["HomeTeam", "AwayTeam", "HomeScore", "AwayScore", "Week"]]

# Engineered two columns
df["HomeResult"] = np.where(df["HomeScore"] > df["AwayScore"], 1, 0)
df["AwayResult"] = np.where(df["AwayScore"] > df["HomeScore"], 1, 0)
df = df[df.Week > 1]


for index, row in df.iterrows():
    df.at[index, 'AwayAverage'] = teams_season_2019.loc[row.AwayTeam,'avg_up_to_week_{0}'.format(row.Week)] if 'avg_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0
    df.at[index, 'HomeAverage'] = teams_season_2019.loc[row.HomeTeam,'avg_up_to_week_{0}'.format(row.Week)] if 'avg_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0

    df.at[index, 'AwayWins'] = teams_season_2019.loc[row.AwayTeam,'wins_up_to_week_{0}'.format(row.Week)] if 'wins_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0
    df.at[index, 'HomeWins'] = teams_season_2019.loc[row.HomeTeam,'wins_up_to_week_{0}'.format(row.Week)] if 'wins_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0

    df.at[index, 'AwayFirstDowns'] = teams_season_2019.loc[row.AwayTeam,'first_downs_up_to_week_{0}'.format(row.Week)] if 'first_downs_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0
    df.at[index, 'HomeFirstDowns'] = teams_season_2019.loc[row.HomeTeam,'first_downs_up_to_week_{0}'.format(row.Week)] if 'first_downs_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0

    df.at[index, 'AwayTime'] = teams_season_2019.loc[row.AwayTeam,'time_of_possession_up_to_week_{0}'.format(row.Week)] if 'time_of_possession_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0
    df.at[index, 'HomeTime'] = teams_season_2019.loc[row.HomeTeam,'time_of_possession_up_to_week_{0}'.format(row.Week)] if 'time_of_possession_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0

    df.at[index, 'AwayThirdDowns'] = teams_season_2019.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] if 'third_down_percentage_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0
    df.at[index, 'HomeThirdDowns'] = teams_season_2019.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] if 'third_down_percentage_up_to_week_{0}'.format(row.Week) in teams_season_2019.columns else 0.0

    
    
    
df

,HomeTeam,AwayTeam,HomeScore,AwayScore,Week,HomeResult,AwayResult,AwayAverage,HomeAverage,AwayWins,HomeWins,AwayFirstDowns,HomeFirstDowns,AwayTime,HomeTime,AwayThirdDowns,HomeThirdDowns
16,CAR,TB,24,35,2,0,1,29.000000,47.000000,0.0,0.0,36.000000,36.000000,29.560000,26.440000,78.900000,78.900000
17,BAL,ARI,40,29,2,1,0,47.000000,102.000000,0.0,1.0,36.000000,54.000000,30.370000,40.700000,70.900000,110.200000
18,CIN,SF,29,71,2,0,1,54.000000,35.000000,1.0,0.0,29.000000,38.000000,30.400000,35.500000,66.700000,69.300000
19,DET,LAC,23,17,2,1,0,52.000000,47.000000,1.0,1.0,43.000000,40.000000,32.500000,39.230000,110.200000,57.700000
20,GB,MIN,36,28,2,1,0,49.000000,17.000000,1.0,1.0,31.000000,23.000000,29.470000,31.300000,86.600000,28.900000
21,HOU,JAX,23,21,2,1,0,45.000000,49.000000,0.0,0.0,31.000000,38.000000,28.450000,29.470000,86.600000,93.200000
22,MIA,NE,0,75,2,0,1,57.000000,17.000000,1.0,0.0,42.000000,21.000000,32.330000,19.530000,86.600000,52.000000
23,NYG,BUF,24,49,2,0,1,29.000000,29.000000,1.0,0.0,40.000000,43.000000,27.590000,27.420000,86.600000,31.500000
24,PIT,SEA,45,49,2,0,1,36.000000,9.000000,1.0,0.0,21.000000,26.000000,24.100000,27.270000,57.700000,43.300000
25,TEN,IND,29,33,2,0,1,42.000000,75.000000,0.0,1.0,38.000000,36.000000,32.900000,29.180000,93.200000,34.700000


In [114]:
df['HomeResult'].value_counts(normalize = True)

1    0.516667
0    0.483333
Name: HomeResult, dtype: float64

In [115]:
df['AwayResult'].value_counts(normalize = True)

0    0.516667
1    0.483333
Name: AwayResult, dtype: float64

In [116]:
# lets try creating a model

# seperate data
selected_features = ['AwayAverage', 'AwayWins', 'AwayFirstDowns', 'AwayTime', 'AwayThirdDowns', 'HomeAverage', 'HomeWins', 'HomeFirstDowns', 'HomeTime', 'HomeThirdDowns']
target = ['HomeResult']

X = df[selected_features].values
y = df[target].values

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=27)

c:\python\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [118]:
y_pred_proba = model.predict_proba(X_test)[:,1]
y_pred_proba

array([0.30106932, 0.31576682, 0.78038597, 0.5377637 , 0.30391797,
       0.33897084, 0.69208983, 0.39004943, 0.3469288 , 0.42410519,
       0.45666366, 0.55402561, 0.51619834, 0.68394631, 0.25460133,
       0.39605276, 0.74285871, 0.56609085, 0.60883748, 0.46996927,
       0.70346731, 0.33185057, 0.68381646, 0.64056898, 0.45784022,
       0.07842127, 0.77649224, 0.8135459 , 0.56562625, 0.78506837,
       0.57680424, 0.04713672, 0.38295555, 0.22831091, 0.67218698,
       0.72828204, 0.57389408, 0.29989539, 0.18728675, 0.23611811,
       0.67986364, 0.69476587, 0.382994  , 0.36463577, 0.2114867 ,
       0.61443516, 0.77016027, 0.94409477, 0.36715927, 0.58087189,
       0.50495153, 0.38130199, 0.51250494, 0.56149609, 0.62726516,
       0.78260542, 0.82237104, 0.21034694, 0.4712985 , 0.86411702])

In [119]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score: %f" % accuracy)


precision = precision_score(y_test, y_pred)
print("Precision Score: %f" % precision)

Accuracy Score: 0.466667
Precision Score: 0.484848


In [120]:
recall = recall_score(y_test, y_pred)
print("Recall Score: %f" % recall)

Recall Score: 0.516129


In [121]:
f1_score(y_test, y_pred)

0.5

In [ ]:
# create new data frame
teams_season_2019 =  pd.DataFrame()
for index, row in team_games_2019.iterrows():
    # ---------------------------------------update each teams score per week------------------------------------------
    teams_season_2019.loc[row.HomeTeam,row.Week] = row.HomeScore
    teams_season_2019.loc[row.AwayTeam,row.Week] = row.AwayScore
    
    teams_season_2019[row.Week] = teams_season_2019[row.Week].fillna(0)

    # ---------------------------------------update each teams games played -------------------------------------------
    if 'GamesPlayed' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] = 0
        teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['GamesPlayed'] = teams_season_2019['GamesPlayed'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'GamesPlayed'] += 1
    teams_season_2019.loc[row.AwayTeam,'GamesPlayed'] += 1
        

    # ----------------------------------------------update score to week------------------------------------------------
    if 'TotalScore' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalScore'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalScore'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalScore'] = teams_season_2019['TotalScore'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalScore'] += row.HomeScore 
    teams_season_2019.loc[row.AwayTeam,'TotalScore'] += row.AwayScore
    
    # get avg points scored up to a certain week
    update_avg_home_score = teams_season_2019.loc[row.HomeTeam,'TotalScore'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_avg_away_score = teams_season_2019.loc[row.AwayTeam,'TotalScore'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'avg_up_to_week_{0}'.format(row.Week + 1)] = update_avg_home_score 
    teams_season_2019.loc[row.AwayTeam,'avg_up_to_week_{0}'.format(row.Week + 1)] = update_avg_away_score
    
    teams_season_2019['avg_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['avg_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalScore'] / (row.Week))
    
    # ----------------------------------- store wins per team up to a certain week ----------------------------------
    if 'TotalWins' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalWins'] = teams_season_2019['TotalWins'].fillna(0)
    # check who won
    if row.HomeScore > row.AwayScore:
        teams_season_2019.loc[row.HomeTeam,'TotalWins'] += 1
    else:
        teams_season_2019.loc[row.AwayTeam,'TotalWins'] += 1
    
    
    teams_season_2019.loc[row.HomeTeam,'wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019.loc[row.HomeTeam,'TotalWins']
    teams_season_2019.loc[row.AwayTeam,'wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019.loc[row.AwayTeam,'TotalWins']
    
    if row.Week >= 4:
        teams_season_2019['wins_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['wins_up_to_week_{0}'.format(row.Week + 1)].fillna(0)
    # ----------------------------------- store first downs per team up to certain week ----------------------------------

    if 'TotalFirstDowns' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] = 0
        teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] = 0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalFirstDowns'] = teams_season_2019['TotalFirstDowns'].fillna(0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] += row.HomeFirstDowns 
    teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] += row.AwayFirstDowns
    
    # get avg points scored up to a certain week
    update_first_downs_home = teams_season_2019.loc[row.HomeTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_first_downs_away = teams_season_2019.loc[row.AwayTeam,'TotalFirstDowns'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'first_downs_up_to_week_{0}'.format(row.Week + 1)] = update_first_downs_home 
    teams_season_2019.loc[row.AwayTeam,'first_downs_up_to_week_{0}'.format(row.Week + 1)] = update_first_downs_away
    
    teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['first_downs_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalFirstDowns'] / (row.Week))
    
    # ----------------------------------- calculate third down % per team up to certain week ----------------------------------

    if 'ThirdDownPercentage' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] = 0.0
        teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] = 0.0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['ThirdDownPercentage'] = teams_season_2019['ThirdDownPercentage'].fillna(0.0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] += row.HomeThirdDownPercentage 
    teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] += row.AwayThirdDownPercentage
    
    # get avg points scored up to a certain week
    update_third_down_percentage_home = teams_season_2019.loc[row.HomeTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_third_down_percentage_away = teams_season_2019.loc[row.AwayTeam,'ThirdDownPercentage'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = update_third_down_percentage_home
    teams_season_2019.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = update_third_down_percentage_away
    
    teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['ThirdDownPercentage'] / (row.Week))
    
    
    # ----------------------------------- calculate average time of possession per team ----------------------------------
    
    away_top = row.AwayTimeOfPossession.split(':')
    home_top = row.HomeTimeOfPossession.split(':')
    away_top = float('{0}.{1}'.format(away_top[0],away_top[1]))
    home_top = float('{0}.{1}'.format(home_top[0],home_top[1]))
    
    
    if 'TotalTimeOfPossession' not in teams_season_2019.columns:
        teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] = 0.0
        teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] = 0.0

    # fill null values with 0 because as scores are appended the column will be NaN
    teams_season_2019['TotalTimeOfPossession'] = teams_season_2019['TotalTimeOfPossession'].fillna(0.0)
    # add score of that week to total score
    teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] += home_top
    teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] += away_top
    
    # get avg points scored up to a certain week
    update_time_of_possession_home = teams_season_2019.loc[row.HomeTeam,'TotalTimeOfPossession'] / teams_season_2019.loc[row.HomeTeam,'GamesPlayed']
    update_time_of_possession_away = teams_season_2019.loc[row.AwayTeam,'TotalTimeOfPossession'] / teams_season_2019.loc[row.AwayTeam,'GamesPlayed']
    teams_season_2019.loc[row.HomeTeam,'time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = update_time_of_possession_home
    teams_season_2019.loc[row.AwayTeam,'time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = update_time_of_possession_away
    
    teams_season_2019['time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = teams_season_2019['time_of_possession_up_to_week_{0}'.format(row.Week + 1)].fillna(teams_season_2019['TotalTimeOfPossession'] / (row.Week ))
    
    
teams_season_2019